# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [8]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

ModuleNotFoundError: No module named 'tensorflow'

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::477094387272:role/service-role/AmazonSageMaker-ExecutionRole-20230910T073521


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://sab-object-detection-udacity/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


In [11]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection-faster-rcnn'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection-faster-rcnn
Sending build context to Docker daemon  728.9MB
Step 1/17 : FROM tensorflow/tensorflow:2.12.0-gpu
2.12.0-gpu: Pulling from tensorflow/tensorflow

f31133a9: Pulling fs layer 
d51e1731: Pulling fs layer 
8ea3b9cc: Pulling fs layer 
a321f215: Pulling fs layer 
e49da50d: Pulling fs layer 
e5648755: Pulling fs layer 
ccdbe31d: Pulling fs layer 
01a2ab97: Pulling fs layer 
5d720362: Pulling fs layer 
75b4be3d: Pulling fs layer 
407e661a: Pulling fs layer 
48faa02b: Pulling fs layer 
99bd4375: Pulling fs layer 
6f8491b9: Pulling fs layer 
1da7cf83: Pulling fs layer 
a3817946: Pulling fs layer 
086451f7: Pulling fs layer 


75b4be3d: Extracting  2.059GB/2.522GBB[18A

59c27a44: Pull complete 134kB/1.134kBDigest: sha256:ce32fe3c53f94938de1513b3fe5fa25aa655a2f4c4bced42cf598fb8ce911219
Status: Downloaded newer image for tensorflow/tensorflow:2.12.0-gpu
 ---> cbe3a4f4c2a0
Step 2/17 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in 5dc92f442320
Removing intermediate container 5dc92f442320
 ---> 925239b143c4
Step 3/17 : RUN rm /etc/apt/sources.list.d/cuda.list
 ---> Running in 66e4db14d201
Removing intermediate container 66e4db14d201
 ---> a209adf5fbec
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in afb68f1842a0
OK
Removing intermediate container afb68f1842a0
 ---> ee0e0e1bb25e
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in eeaf43158b13
Executing: /tmp/apt-key-gpghome.Co0RFJYglb/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
gpg: requesting key from 'https://developer.download.nvidia.co

The following NEW packages will be installed:
  blt fontconfig-config fonts-dejavu-core git git-man less libbsd0 libcairo2
  libcbor0.6 libcurl3-gnutls libdrm-amdgpu1 libdrm-common libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libdrm2 libedit2 libegl-dev libegl-mesa0
  libegl1 liberror-perl libfido2-1 libfontconfig1 libfreetype6 libgbm1
  libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common libgl-dev
  libgl1 libgl1-mesa-dev libgl1-mesa-dri libglapi-mesa libgles-dev libgles1
  libgles2 libglvnd-dev libglvnd0 libglx-dev libglx-mesa0 libglx0
  libimagequant0 libjbig0 libjpeg-turbo8 libjpeg8 liblcms2-2 libllvm12
  libopengl-dev libopengl0 libpciaccess0 libpixman-1-0 libpng16-16
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
  libpthread-stubs0-dev libsensors-config libsensors5 libtcl8.6 libtiff5
  libtk8.6 libvulkan1 libwayland-client0 libwayland-server0 libwebp6
  libwebpdemux2 libwebpmux3 libx11-6 libx11-data libx11-dev libx11-xcb1
  libxau-dev libxau6 libxcb

Get:64 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libllvm12 amd64 1:12.0.0-3ubuntu1~20.04.5 [18.8 MB]
Get:65 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors-config all 1:3.6.0-2ubuntu1.1 [6052 B]
Get:66 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors5 amd64 1:3.6.0-2ubuntu1.1 [27.2 kB]
Get:67 http://archive.ubuntu.com/ubuntu focal/main amd64 libvulkan1 amd64 1.2.131.2-1 [93.3 kB]
Get:68 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1-mesa-dri amd64 21.2.6-0ubuntu0.1~20.04.2 [11.0 MB]
Get:69 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx-mesa0 amd64 21.2.6-0ubuntu0.1~20.04.2 [137 kB]
Get:70 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx0 amd64 1.3.2-1~ubuntu0.20.04.2 [32.5 kB]
Get:71 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1 amd64 1.3.2-1~ubuntu0.20.04.2 [85.8 kB]
Get:72 http://archive.ubuntu.com/ubuntu focal/main amd64 xorg-sgml-doctools all 1:1.11-1 [12.

...

  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling tensorboard-2.12.0:
      Successfully uninstalled tensorboard-2.12.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
      Successfully uninstalled tensorflow-2.12.0
Removing intermediate container 7a7aac7b4851
 ---> 116a0365e205
Step 15/17 : ENV TF_CPP_MIN_LOG_LEVEL 3
 ---> Running in 0ac480575ad4
Removing intermediate container 0ac480575ad4
 ---> 9eac5852d8f4
Step 16/17 : RUN pip3 install sagemaker-training
 ---> Running in 95354ac8814a
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 21.8 MB/s eta 0:00:00
  Created wheel for sagemaker-training: filename=sagemaker_training-4.7.0-cp38-cp38-linux_x86_64.whl size=93409 sha256=003d2a8989f422736db50b13b59e18fd1fd1127773b7d07cd06fd4a394389f53
  Stored in directory: /root/.cache/pip/wheels/16/76/fb/32f317f8d2ea470479580d8d5dcccc1bfd7994b273df803aaf
  Created wheel for inotify_simple: filename=inotify_simple-1.2.1-py3-none-any.whl size=8201 sha256=f90a747f0aec21bde27fbe14a64e3a50ccae821f1390aadb4efb7aca85a79b70
  Stored in directory: /root/.cache/pip/wheels/8b/2d/c2/46bac8503a2469

f5e16b0: Pushing  847.8MB/3.191GBPushing  260.2MB/323.3MB

f5e16b0: Pushing  2.626GB/3.191GB

f5e16b0: Pushed   3.224GB/3.191GB20230926060559: digest: sha256:2bccb6da210d6575124d90d09abcc5ec4a409d93dc93c4579615e08957ec85bc size: 6605
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [12]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

477094387272.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection-faster-rcnn:20230926060559


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [7]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/faster-rcnn-resnet50-v1.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster-rcnn-resnet50-v1.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint

--2023-09-22 06:32:41--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.62.207, 172.253.115.207, 172.253.122.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.62.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘/tmp/faster-rcnn-resnet50-v1.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 5.65M 36s
    50K .......... .......... .......... .......... ..........  0% 59.4M 20s
   100K .......... .......... .......... .......... ..........  0% 15.0M 18s
   150K .......... .......... .......... .......... ..........  0% 21.4M 16s
   200K .......... .......... .......... .......... ..........  0%  135M 13s
   250K .......... .......... .......... .......... ..........  0% 47.8M 11s
   300K .......... .......... .......

  5100K .......... .......... .......... .......... ..........  2% 55.4M 6s
  5150K .......... .......... .......... .......... ..........  2% 53.0M 5s
  5200K .......... .......... .......... .......... ..........  2% 69.1M 5s
  5250K .......... .......... .......... .......... ..........  2%  210M 5s
  5300K .......... .......... .......... .......... ..........  2% 55.7M 5s
  5350K .......... .......... .......... .......... ..........  2% 55.3M 5s
  5400K .......... .......... .......... .......... ..........  2% 57.5M 5s
  5450K .......... .......... .......... .......... ..........  2% 62.9M 5s
  5500K .......... .......... .......... .......... ..........  2%  181M 5s
  5550K .......... .......... .......... .......... ..........  2% 53.9M 5s
  5600K .......... .......... .......... .......... ..........  2% 55.7M 5s
  5650K .......... .......... .......... .......... ..........  2% 64.4M 5s
  5700K .......... .......... .......... .......... ..........  2% 62.5M 5s
  5750K ....

 10500K .......... .......... .......... .......... ..........  5%  147M 6s
 10550K .......... .......... .......... .......... ..........  5%  132M 6s
 10600K .......... .......... .......... .......... ..........  5% 76.8M 6s
 10650K .......... .......... .......... .......... ..........  5% 91.2M 6s
 10700K .......... .......... .......... .......... ..........  5%  113M 5s
 10750K .......... .......... .......... .......... ..........  5%  370M 5s
 10800K .......... .......... .......... .......... ..........  5% 63.9M 5s
 10850K .......... .......... .......... .......... ..........  5%  129M 5s
 10900K .......... .......... .......... .......... ..........  5%  110M 5s
 10950K .......... .......... .......... .......... ..........  5% 48.0M 5s
 11000K .......... .......... .......... .......... ..........  5%  287M 5s
 11050K .......... .......... .......... .......... ..........  5%  382M 5s
 11100K .......... .......... .......... .......... ..........  5%  203M 5s
 11150K ....

 15900K .......... .......... .......... .......... ..........  7%  361M 4s
 15950K .......... .......... .......... .......... ..........  7%  315M 4s
 16000K .......... .......... .......... .......... ..........  7%  354M 4s
 16050K .......... .......... .......... .......... ..........  7%  297M 4s
 16100K .......... .......... .......... .......... ..........  7%  376M 4s
 16150K .......... .......... .......... .......... ..........  7%  322M 4s
 16200K .......... .......... .......... .......... ..........  7%  344M 4s
 16250K .......... .......... .......... .......... ..........  7%  266M 4s
 16300K .......... .......... .......... .......... ..........  7%  359M 4s
 16350K .......... .......... .......... .......... ..........  7% 3.99M 4s
 16400K .......... .......... .......... .......... ..........  7%  141M 4s
 16450K .......... .......... .......... .......... ..........  7%  159M 4s
 16500K .......... .......... .......... .......... ..........  7%  149M 4s
 16550K ....

 21300K .......... .......... .......... .......... .......... 10%  164M 4s
 21350K .......... .......... .......... .......... .......... 10%  212M 4s
 21400K .......... .......... .......... .......... .......... 10%  349M 4s
 21450K .......... .......... .......... .......... .......... 10%  187M 4s
 21500K .......... .......... .......... .......... .......... 10%  321M 4s
 21550K .......... .......... .......... .......... .......... 10%  216M 4s
 21600K .......... .......... .......... .......... .......... 10%  338M 4s
 21650K .......... .......... .......... .......... .......... 10%  214M 4s
 21700K .......... .......... .......... .......... .......... 10%  233M 4s
 21750K .......... .......... .......... .......... .......... 10%  295M 4s
 21800K .......... .......... .......... .......... .......... 10%  367M 4s
 21850K .......... .......... .......... .......... .......... 10%  299M 4s
 21900K .......... .......... .......... .......... .......... 10% 8.72M 4s
 21950K ....

...

188700K .......... .......... .......... .......... .......... 91% 93.8M 0s
188750K .......... .......... .......... .......... .......... 91% 57.5M 0s
188800K .......... .......... .......... .......... .......... 91%  150M 0s
188850K .......... .......... .......... .......... .......... 91% 66.8M 0s
188900K .......... .......... .......... .......... .......... 91% 61.1M 0s
188950K .......... .......... .......... .......... .......... 91% 54.9M 0s
189000K .......... .......... .......... .......... .......... 91% 60.8M 0s
189050K .......... .......... .......... .......... .......... 91% 37.1M 0s
189100K .......... .......... .......... .......... .......... 91% 32.6M 0s
189150K .......... .......... .......... .......... .......... 91% 19.0M 0s
189200K .......... .......... .......... .......... .......... 91% 98.0M 0s
189250K .......... .......... .......... .......... .......... 91%  311M 0s
189300K .......... .......... .......... .......... .......... 91% 21.5M 0s
189350K ....

194100K .......... .......... .......... .......... .......... 93% 83.6M 0s
194150K .......... .......... .......... .......... .......... 93%  308M 0s
194200K .......... .......... .......... .......... .......... 93%  193M 0s
194250K .......... .......... .......... .......... .......... 93% 81.7M 0s
194300K .......... .......... .......... .......... .......... 93%  345M 0s
194350K .......... .......... .......... .......... .......... 93% 70.0M 0s
194400K .......... .......... .......... .......... .......... 93%  317M 0s
194450K .......... .......... .......... .......... .......... 93%  362M 0s
194500K .......... .......... .......... .......... .......... 93%  350M 0s
194550K .......... .......... .......... .......... .......... 93% 51.7M 0s
194600K .......... .......... .......... .......... .......... 94%  343M 0s
194650K .......... .......... .......... .......... .......... 94% 73.3M 0s
194700K .......... .......... .......... .......... .......... 94%  342M 0s
194750K ....

199500K .......... .......... .......... .......... .......... 96%  278M 0s
199550K .......... .......... .......... .......... .......... 96% 59.4M 0s
199600K .......... .......... .......... .......... .......... 96% 47.7M 0s
199650K .......... .......... .......... .......... .......... 96%  341M 0s
199700K .......... .......... .......... .......... .......... 96%  236M 0s
199750K .......... .......... .......... .......... .......... 96% 78.6M 0s
199800K .......... .......... .......... .......... .......... 96%  199M 0s
199850K .......... .......... .......... .......... .......... 96% 43.4M 0s
199900K .......... .......... .......... .......... .......... 96%  315M 0s
199950K .......... .......... .......... .......... .......... 96% 78.7M 0s
200000K .......... .......... .......... .......... .......... 96%  279M 0s
200050K .......... .......... .......... .......... .......... 96% 59.2M 0s
200100K .......... .......... .......... .......... .......... 96%  329M 0s
200150K ....

204900K .......... .......... .......... .......... .......... 98% 74.1M 0s
204950K .......... .......... .......... .......... .......... 99%  221M 0s
205000K .......... .......... .......... .......... .......... 99%  328M 0s
205050K .......... .......... .......... .......... .......... 99%  315M 0s
205100K .......... .......... .......... .......... .......... 99%  371M 0s
205150K .......... .......... .......... .......... .......... 99%  279M 0s
205200K .......... .......... .......... .......... .......... 99% 21.8M 0s
205250K .......... .......... .......... .......... .......... 99%  338M 0s
205300K .......... .......... .......... .......... .......... 99%  332M 0s
205350K .......... .......... .......... .......... .......... 99% 42.4M 0s
205400K .......... .......... .......... .......... .......... 99%  332M 0s
205450K .......... .......... .......... .......... .......... 99%  354M 0s
205500K .......... .......... .......... .......... .......... 99% 59.9M 0s
205550K ....

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [13]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection-frcnn'
)

estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: tf2-object-detection-frcnn-2023-09-26-06-20-41-788


2023-09-26 06:20:51 Starting - Starting the training job...
2023-09-26 06:21:17 Starting - Preparing the instances for training.........
2023-09-26 06:22:33 Downloading - Downloading input data...
2023-09-26 06:23:08 Training - Downloading the training image...............
2023-09-26 06:25:29 Training - Training image download completed. Training in progress...2023-09-26 06:25:57,825 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-26 06:25:57,859 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-26 06:25:57,894 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-09-26 06:25:57,907 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "cur

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0926 06:26:12.842206 139821553157952 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0926 06:26:15.607884 139821553157952 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/Fa

INFO:tensorflow:Step 200 per-step time 0.170s
I0926 06:28:27.053987 139821553157952 model_lib_v2.py:705] Step 200 per-step time 0.170s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.30408925,
 'Loss/BoxClassifierLoss/localization_loss': 0.4586568,
 'Loss/RPNLoss/localization_loss': 0.5256813,
 'Loss/RPNLoss/objectness_loss': 0.054931022,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.3433584,
 'learning_rate': 0.0159997}
I0926 06:28:27.054310 139821553157952 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.30408925,
 'Loss/BoxClassifierLoss/localization_loss': 0.4586568,
 'Loss/RPNLoss/localization_loss': 0.5256813,
 'Loss/RPNLoss/objectness_loss': 0.054931022,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.3433584,
 'learning_rate': 0.0159997}
INFO:tensorflow:Step 300 per-step time 0.170s
I0926 06:28:44.067683 139821553157952 model_lib_v2.py:705] Step 300 per-step time 0.170s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classificati

INFO:tensorflow:Step 1200 per-step time 0.173s
I0926 06:31:19.231815 139821553157952 model_lib_v2.py:705] Step 1200 per-step time 0.173s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.19900975,
 'Loss/BoxClassifierLoss/localization_loss': 0.27454543,
 'Loss/RPNLoss/localization_loss': 0.30856135,
 'Loss/RPNLoss/objectness_loss': 0.07249333,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.85460985,
 'learning_rate': 0.0293332}
I0926 06:31:19.232141 139821553157952 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.19900975,
 'Loss/BoxClassifierLoss/localization_loss': 0.27454543,
 'Loss/RPNLoss/localization_loss': 0.30856135,
 'Loss/RPNLoss/objectness_loss': 0.07249333,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.85460985,
 'learning_rate': 0.0293332}
INFO:tensorflow:Step 1300 per-step time 0.170s
I0926 06:31:36.213318 139821553157952 model_lib_v2.py:705] Step 1300 per-step time 0.170s
INFO:tensorflow:{'Loss/BoxClassifierLoss/clas

/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
INFO:tensorflow:depth of additional conv before box predictor: 0
I0926 06:33:58.939584 140301618825024 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.
W0926 06:34:06.251935 140301618825024 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:460: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
Instructions for updating:
Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.
See `

INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0926 06:38:52.466540 140301618825024 checkpoint_utils.py:168] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Timed-out waiting for a checkpoint.
I0926 06:39:01.480874 140301618825024 checkpoint_utils.py:231] Timed-out waiting for a checkpoint.
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.15s).
Accumulating evaluation results...
DONE (t=0.45s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.210
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.331
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.502
 Ave

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your writeup goes here.